In [1]:
import pandas as pd
import torch
from torch.utils.data import DataLoader
import torch.optim as optim
import torch.nn as nn
from tqdm import tqdm
from featout_exp.utils import CustomDataset, load_model

In [7]:
# Reading the dataframe
df = pd.read_csv('csvs/second_half.csv', index_col= 0).iloc[:100]

In [10]:
dataset = CustomDataset(dataframe=df)

# Create DataLoader to handle batching
batch_size = 32  # You can modify this as needed
test_loader = DataLoader(dataset, batch_size=batch_size, shuffle= False)

In [11]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model = load_model(device)

In [12]:
model.eval()  # Set the model to evaluation mode

all_predictions = []
all_true_labels = []
loss_per_image = []
criterion = nn.CrossEntropyLoss(reduction='none')  # Using 'none' to compute loss for each image

with torch.no_grad():  # Disable gradient calculation
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        # Forward pass
        outputs = model(inputs)
        
        # Calculate loss for each image in the batch
        losses = criterion(outputs, labels)
        loss_per_image.extend(losses.cpu().numpy())
        
        # Get predicted labels
        _, predicted = torch.max(outputs.data, 1)
        all_predictions.extend(predicted.cpu().numpy())
        all_true_labels.extend(labels.cpu().numpy())


In [14]:
test_df = df

In [15]:
test_df["loss"] = loss_per_image
test_df["pred_true"] =all_predictions